<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/7_2_bond_duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

7.　債券投資理論とデュレーション

　デュレーションの算出は、償還期間n、クーポンレートci、複利最終利回りr、額面価格Nを定義式に与えることで算出可能です。一方で、即時グロス決済に要する営業日数、年間営業日の設定など、実務の現場で応用するには細かい設定が必要となります。このため本例では数理ファイナンス実務のために開発されたオープンソース・ソフトウェア・ライブラリのQuantLibを用いて実装を行います。まず、このQuantLibソフトウェアをGoogle Colab上で、ひとつのコードセルのみを用いてインストールします。

In [ ]:
pip install QuantLib

　QuantLibがインストールされれば、次にこのライブラリを数値演算ライブラリのnumpyとともにインポートします。

In [ ]:
#[1]ライブラリの読み込み
import QuantLib as ql#数理ファイナンス・ライブラリのインポート
import numpy as np

　次に、償還期間n、クーポンレートci、複利最終利回りr、額面価格Nの4つのデータを与えます。本例は確定利付債のため、ciは期間を通じてすべて等しい支払（受取）額となります。本例では令和4年度個人向け国債新窓販国債367回債（償還期間10年）を参考に、10年債の4条件の設定を下記の通りとしました。

In [ ]:
#[2]償還期間・クーポーン・利回りの設定
#368回債の場合(2022年11月8日発行)
n = 3650#償還期間10年  
c = 0.002#クーポン・レート
r = 0.00248#複利最終利回り
N = 100#額面価格

　続いて発行日と償還期間を設定します。発行日を「本日」として、q.l.Dateにより[2]の設定が日次データであることを定義します。厳密に国債367回債の発行日を指定する場合は、start = ql.Date(8,11,2022)
と記述してください。 q.Periodは時間データの頻度を定義する関数であり、これを用いて「本日」を基準とする日付に償還日数加えることで、”maturity”を定義します。


In [ ]:
#[3]発行日・償還期間の設定
start = ql.Date(8,11,2022)#厳密に発行日を記入する
maturity = start + ql.Period(n, ql.Days)#[2]nを用い償還期間を設定

　最後に確定利付債のための関数であるql.FixedRateBond()を用いてデュレーションを算出します。ql.FixedRateBondの（）内には左から順に、決済日数、決済手段、額面価格、発行日、償還期間、時間頻度、クーポン・レート、年間の営業日数を入力します。TARGETはTrans-European Automated Real-time Gross Settlement Express Transferの略です。本例の決済手段はこれを入力し、債券決済期間として2営業日（T+1）を入力しています。利回り（rate）の場合はql.InterestRate()を用い、最終利回り、年間営業日数、複利か否か、年率か否か、を順に指示します。年間営業日数は365日ql.Actual365Fixed()と360日ql.Actual365Fixed()いずれの算出方法も指定可能ですが、本例では前者を用いています。

In [ ]:
#[4]デュレーションの算出
bond = ql.FixedRateBond(2, ql.TARGET(), N, start, maturity, ql.Period(n,ql.Days), [c], ql.Actual365Fixed())#債券の決済期間・額面・発行・償還日・頻度他を順に設定
rate = ql.InterestRate(r, ql.Actual365Fixed(), ql.Compounded, ql.Annual)#利回り・年間営業日数・複利・年間利回りを順に設定
norm_duration = ql.BondFunctions.duration(bond, rate, ql.Duration.Simple)#デュレーションの算出
mod_duration = ql.BondFunctions.duration(bond, rate, ql.Duration.Modified)#修正デュレーションの算出
mac_duration = ql.BondFunctions.duration(bond, rate, ql.Duration.Macaulay)#マコーレー・デュレーションの算出

　本例の10年債の修正デュレーションとマコーレー・デュレーションの算出結果は下記の通りです。上記のsource codeに令和4年度個人向け国債第147回債（償還期間3年）の4条件を[3]で与えると、デュレーションはより小さくなります。

In [ ]:
#[5]デュレーション算出結果の表示
print('normal_duration:{}'.format(np.round(norm_duration,decimals=3)))#算出結果の表示
print('modified_duration:{}'.format(np.round(mod_duration,decimals=3)))#算出結果の表示
print('macaulay_duration:{}'.format(np.round(mac_duration,decimals=3)))

normal_duration:9.83
modified_duration:9.806
macaulay_duration:9.83
